In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai import *
from fastai.vision import *
import torch
from torch import nn
from dsin.ae.data_manager.data_loader import (
    SideinformationImageImageList, ImageSiTuple)
from dsin.ae import config
from dsin.ae.si_ae import SideInformationAutoEncoder
from dsin.ae.si_net import SiNetChannelIn
from dsin.ae.loss_man import LossManager
from dsin.ae.distortions import Distortions
from dsin.ae.kitti_normalizer import ChangeImageStatsToKitti, ChangeState
from dsin.ae import config


In [3]:
# config.use_si_flag = SiNetChannelIn.WithSideInformation
config.use_si_flag

<SiNetChannelIn.NoSideInformation: 3>

In [4]:
torch.__version__

'1.4.0'

In [5]:
si_autoencoder = SideInformationAutoEncoder(config.use_si_flag)


In [6]:
ls ../src/dsin

ae/  data/  dev_nb/  version.py


In [7]:
image_list = SideinformationImageImageList.from_csv(path="../src/dsin/data",csv_names=["KITTI_stereo_train.txt","KITTI_stereo_val.txt"])

In [8]:
image_list.items[0]

'../src/dsin/data/data_scene_flow_multiview/training/image_3/000000_10.png'

In [9]:
ll = image_list.split_by_valid_func(lambda x: 'testing'  in x ).label_from_func(lambda x: x)

In [10]:
batchsize = 1
data = (image_list
        .split_by_valid_func(lambda x: 'testing'  in x )
        .label_from_func(lambda x: x)
        .transform(None,size=(336,1224),resize_method=ResizeMethod.CROP,tfm_y=True)
        .databunch(bs=batchsize))
 

In [11]:
# data.train_ds[0][0]

In [12]:
my_learner = Learner(data=data,
                     model=si_autoencoder,
                     opt_func=torch.optim.Adam,
                     loss_func=LossManager(config.use_si_flag))

In [13]:
 my_learner.fit(1)


epoch,train_loss,valid_loss,time
0,419.649933,417.744232,08:21


In [14]:
# my_learner.lr_find()
my_learner.save('stage-1')

In [15]:
# my_learner.recorder.plot()
my_learner.load('stage-1')

Learner(data=ImageDataBunch;

Train: LabelList (1576 items)
x: SideinformationImageImageList
ImageSiTuple [tensor([[[0.0866, 0.0857, 0.0909,  ..., 1.0000, 1.0000, 1.0000],
         [0.1137, 0.1107, 0.1078,  ..., 1.0000, 1.0000, 1.0000],
         [0.1178, 0.1198, 0.1188,  ..., 1.0000, 1.0000, 1.0000],
         ...,
         [0.1303, 0.1213, 0.1057,  ..., 0.0451, 0.0490, 0.0491],
         [0.1244, 0.1155, 0.1029,  ..., 0.0460, 0.0500, 0.0471],
         [0.1216, 0.1105, 0.0990,  ..., 0.0469, 0.0509, 0.0501]],

        [[0.0950, 0.0900, 0.0996,  ..., 1.0000, 1.0000, 1.0000],
         [0.1009, 0.1069, 0.1208,  ..., 1.0000, 1.0000, 1.0000],
         [0.1108, 0.1217, 0.1274,  ..., 1.0000, 1.0000, 1.0000],
         ...,
         [0.1470, 0.1549, 0.1588,  ..., 0.0560, 0.0511, 0.0520],
         [0.1549, 0.1599, 0.1637,  ..., 0.0607, 0.0628, 0.0599],
         [0.1569, 0.1608, 0.1617,  ..., 0.0656, 0.0726, 0.0688]],

        [[0.1020, 0.1146, 0.1118,  ..., 1.0000, 1.0000, 1.0000],
         [0.1077

In [17]:
my_learner.save("trained_model-stage1", return_path=True)


PosixPath('../src/dsin/data/models/trained_model-stage1.pth')

In [29]:
! ls -l ../src/dsin/data/

total 61688
-rwxr-xr-x 1 jupyter jupyter      334 Jul  9 17:31 create_shared_folder_symlink.sh
drwxr-xr-x 4 jupyter jupyter     4096 Jul 10 07:21 data_scene_flow_multiview
-rw-r--r-- 1 jupyter jupyter     1700 Jul 10 07:25 data_shifter.py
drwxr-xr-x 4 jupyter jupyter     4096 Jul  9 18:19 data_stereo_flow_multiview
drwxr-xr-x 3 jupyter jupyter     4096 Jul  9 17:31 example
-rw-r--r-- 1 jupyter jupyter        0 Jul  9 17:31 __init__.py
-rwxr-xr-x 1 jupyter jupyter   407536 Jul  9 17:31 KITTI_general_test.txt
-rwxr-xr-x 1 jupyter jupyter 19944504 Jul  9 17:31 KITTI_general_train.txt
-rwxr-xr-x 1 jupyter jupyter   103116 Jul  9 17:31 KITTI_general_val.txt
-rwxr-xr-x 1 jupyter jupyter    89260 Jul  9 17:31 KITTI_stereo_test.txt
-rwxr-xr-x 1 jupyter jupyter   181216 Jul  9 17:31 KITTI_stereo_train.txt
-rwxr-xr-x 1 jupyter jupyter    89260 Jul  9 17:31 KITTI_stereo_val.txt
drwxr-xr-x 2 jupyter jupyter     4096 Jul 10 07:46 models
-rwxr-xr-x 1 jupyter jupyter      223 Jul  9 17:31 nano_KITTI_

In [26]:
my_learner.export("trained_model_ex_stage1.pkl")

In [28]:
my_learner.path

PosixPath('../src/dsin/data')

In [16]:
# my_learner.fit_one_cycle(10)
